In [14]:
# 教學影片 AutoGen Quickstart - 5. Teaching Agents to Use Pre-Written Programs
# 1. Functions Basics.mp4

In [1]:
import autogen
from app import list_datasets

# 20240330 不懂 gpt-3.5-turbo 會出現錯誤，gpt-3.5-turbo-1106、gpt-3.5-turbo-16k 就不會 ????
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-16k"]
    }
)

llm_config_gpt35 = {
    "config_list" : config_list_gpt35,
    "temperature" : 0.2,
    "timeout" : 300,
    "seed" : 4,
    "functions": [{
	"name": "list_datasets",
	"description": "print out the titles ans URLs of datasets list on the main dataset catalog page",
	"parameters": {
		"type": "object",
		"properties": {
			"url": {
				"type": "string",
				"description": "url of the page.",  # 重要，提供給 agent 的訊息，讓他們知道如何使用該功能
				}
			},
		"required": ["url"],
	},
	}]
}

In [2]:
# 修改 system_message，以便任務達成後盡快結束對話。
assistant = autogen.AssistantAgent(
    name="assistant",
    system_message="""You are a helpful assistant. Once the purpose is achieved, Add a new line with the content of "終止, TERMINATE" at the end.""",
    llm_config=llm_config_gpt35
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",  # NEVER, TERMINATE, ALWAYS
    max_consecutive_auto_reply=6,
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "agent_work_dir",
        "use_docker": False,
    },
    llm_config = llm_config_gpt35
)

In [3]:
# 接著要註冊函數，以下是註冊函數的方法之一，function_map，是一個字典，用於將函數名映射到相應的函數對象上。不懂就是不懂，啥天書?
user_proxy.register_function(
	function_map={
		"list_datasets": list_datasets
	}
)

In [4]:
user_proxy.initiate_chat(
    assistant, 
    message = """Run the 'list_datasets' function and tell me which datasets would be useful for someone who wants
                 to write an article about public health. And tell me the reason why pick it in traditional chinese for each picked dataset. 
                 'https://catalog.data.gov/dataset' is the url."""
)


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


user_proxy (to assistant):

Run the 'list_datasets' function and tell me which datasets would be useful for someone who wants
                 to write an article about public health. And tell me the reason why pick it in traditional chinese for each picked dataset. 
                 'https://catalog.data.gov/dataset' is the url.

--------------------------------------------------------------------------------


No default IOStream has been set, defaulting to IOConsole.
No default IOStream has been set, defaulting to IOConsole.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}